### Settings

In [1]:
which_data = 'g'
# 'c' for color images
# 'g' for grayscale
# 'f' for fourier


take_subset = False
# True to only use 2 classes
# False to use all 10
# note capitalization

do_PCA = True
components = 200
# True will run PCA to reduce the dimension before doing diffusion/LE
# False will skip this step
#
# if take_subset was set to False, I recommend doing PCA
#
# components is the number of components you get from PCA

## File Path
data_path = "C:\\Users\\ecwit\\Data\\cifar-10" 

## Import libraries

In [2]:
exec(open("auxiliary\\imports.py").read())
%matplotlib inline

## Import raw data

In [3]:
exec(open("auxiliary\\loadCFAR.py").read())
## data: raw_images
## classes: cls
## index_labels: indices for each class

# for i in range(10):
#     print(index_labels[:, i])

Loading data: C:\Users\ecwit\Data\cifar-10\data_batch_1
Loading data: C:\Users\ecwit\Data\cifar-10\data_batch_2
Loading data: C:\Users\ecwit\Data\cifar-10\data_batch_3
Loading data: C:\Users\ecwit\Data\cifar-10\data_batch_4
Loading data: C:\Users\ecwit\Data\cifar-10\data_batch_5


## Convert original arrays to correct 3072 dim array per image

In [4]:
images = raw_images

del raw_images

images=images.ravel()

if which_data == 'c':
    images = images.reshape(-1, 32*32*3)
else:
    images = images.reshape(-1, 32*32)
    
np.random.seed(5)
sub_ind=np.random.choice(len(images),40000,replace=False)

labels_subset=cls[np.where(sub_ind)]
reduced = images[np.where(sub_ind)]

## Do Diffusion Map Time Complexity

In [5]:
# sizes=[100,1000,5000,10000, 20000,40000]
sizes = [100, 1000,5000,10000, 20000,40000]
process_time=np.zeros((len(sizes), 2))

for i in range(len(sizes)):
    
    num_coord = 3
    neighbors = 35
    eps = 12000
    alph = 1

    doPlot = False
    doKNN = False
    doSVM = False

    data = reduced[0:sizes[i]]
    classes = labels_subset[0:sizes[i]]
    
    title = "Diffusion Coordinates, CIFAR grayscale"
    
#     print("FOR " + str(i) + " COMPONENTS")

    start=datetime.datetime.now()
    exec(open("auxiliary\\doDiffusion.py").read())
    end=datetime.datetime.now()
    process_time[i, 0] = int(sizes[i])
    process_time[i, 1] = (end-start).total_seconds()
    
for i in range(len(sizes)):
    print(str(int(sizes[i])) + "    " + str(process_time[i, 1]))

Diffusion Map Time Taken: 0:00:00.168547


Diffusion Map Time Taken: 0:00:00.559340


Diffusion Map Time Taken: 0:00:09.961605


Diffusion Map Time Taken: 0:00:40.955663


Diffusion Map Time Taken: 0:02:41.903433


Diffusion Map Time Taken: 0:11:01.765438


100    0.171539
1000    0.561306
5000    9.964599
10000    40.957658
20000    161.907423
40000    661.768428


## LAPLACIAN EIGENMAPS

In [6]:
# sizes=[100,1000,5000,10000, 20000,40000]
sizes = [100, 1000,5000,10000, 20000,40000]
process_time=np.zeros((len(sizes), 2))

for i in range(len(sizes)):
    
    num_coord = 3
    neighbors = 50

    gam = .00001
    # the gamma for LE is 1/epsilon from DM
    # unused if we choose 0 and 1 weights

    doPlot = False
    doKNN = False
    doSVM = False

    data = reduced[0:sizes[i]]
    classes = labels_subset[0:sizes[i]]
    
    #del images
    #del raw_images
    title = "Laplacian Eigenmap Coordinates, CIFAR Grayscale"
    
    start=datetime.datetime.now()
    exec(open("auxiliary\\doLE.py").read())
    end=datetime.datetime.now()
    process_time[i, 0] = int(sizes[i])
    process_time[i, 1] = (end-start).total_seconds()
#     print("#############################\n\n")


for i in range(len(sizes)):
    print(str(int(sizes[i])) + "    " + str(process_time[i, 1]))

Laplacian Eigenmap Time Taken: 0:00:00.013964


Laplacian Eigenmap Time Taken: 0:00:02.459827


Laplacian Eigenmap Time Taken: 0:01:02.164807


Laplacian Eigenmap Time Taken: 0:04:10.283904


Laplacian Eigenmap Time Taken: 0:19:57.883814


Laplacian Eigenmap Time Taken: 1:54:34.907572


100    0.015956
1000    2.461823
5000    62.166801
10000    250.286894
20000    1197.886815
40000    6874.950457
